In [ ]:
import os
os.environ['CUDA_LAUNCH_BLOCKING'] = "1"
os.environ["CUDA_VISIBLE_DEVICES"] = "0"

In [ ]:
# !pip install torchaudio

In [ ]:
# import torch
# import torchaudio

In [ ]:
conda create -n simpletransformers python pandas tqdm -y

In [ ]:
conda activate simpletransformers

In [ ]:
conda install pytorch cpuonly -c pytorch

In [1]:
!pip install -U transformers==4.7.0 datasets scipy scikit-learn

In [2]:
import matplotlib.pyplot as plt
import random
import logging
from IPython.display import display, HTML

import numpy as np
import pandas as pd
import datasets
from datasets import Dataset, load_dataset, load_metric, ClassLabel, Sequence
from transformers import AutoTokenizer, AutoModelForSequenceClassification, TrainingArguments, Trainer
from sklearn.model_selection import train_test_split

from sklearn.metrics import precision_recall_fscore_support

# TRAIN_FILE_PATH = '/content/drive/MyDrive/mini_project/english_news/English_news_data/en_news_with_health.csv'
TRAIN_FILE_PATH = '/kaggle/input/english-data/en_final_dataset.csv'
# TRAIN_FILE_PATH = '/kaggle/input/en-data-health/en_news_with_health.csv'


In [3]:
data = pd.read_csv(TRAIN_FILE_PATH)

data = data.rename(columns={'topic_idx':'labels'})
data['labels'] = data['labels'].astype('int')

data_dummies = pd.get_dummies(data, columns=['labels'])

data_dummies.head()

In [5]:
data = data_dummies.drop(['Unnamed: 0', 'title_len'], axis=1)
data

In [5]:
# import re

# def preprocessing(text):
#   # 개행문자 제거
#   text = re.sub('\\\\n', ' ', text)

#   # 한글 제거
#   text = re.sub('[가-힣ㄱ-ㅎㅏ-ㅣ]', ' ', text)

#   # 중복으로 생성된 공백값 제거
#   text = re.sub('[\s]+', ' ', text)

#   # 영문자를 소문자로 만듦
#   text = text.lower()
#   return text

In [6]:
# data['title'] = data['title'].map(preprocessing)

In [5]:
data

In [6]:
# dataset_train = Dataset.from_pandas(data)

data['labels'] = list(zip(data.labels_0.tolist(), data.labels_1.tolist(), data.labels_2.tolist(), data.labels_3.tolist(),  data.labels_4.tolist(), data.labels_5.tolist(), data.labels_6.tolist()))
data['text'] = data['title'].apply(lambda x: x.replace('\n', ' '))

data.head()

# dataset_train, dataset_validation = dataset_train.train_test_split(test_size=0.2).values()
# datasets = dataset_train, dataset_validation

# datasets

In [7]:
from sklearn.model_selection import train_test_split


train_df, eval_df = train_test_split(data, test_size=0.2)

In [8]:
!pip install simpletransformers

In [9]:
from simpletransformers.classification import MultiLabelClassificationModel

# model = MultiLabelClassificationModel('roberta', 'roberta-base', num_labels=7, args={'train_batch_size':2, 'gradient_accumulation_steps':16, 'learning_rate': 2e-5, 'num_train_epochs': 3, 'max_seq_length': 100})
# model = MultiLabelClassificationModel('roberta', 'distilroberta-base', num_labels=7, args={'train_batch_size':2, 'gradient_accumulation_steps':16, 'learning_rate': 3e-5, 'num_train_epochs': 3, 'max_seq_length': 100}) # 91.1
# model = MultiLabelClassificationModel('roberta', 'distilroberta-base', num_labels=7, args={'train_batch_size':2, 'gradient_accumulation_steps':2, 'learning_rate': 3e-5, 'num_train_epochs': 3, 'max_seq_length': 100}) # 90.2
model = MultiLabelClassificationModel('roberta', 'distilroberta-base', num_labels=7, args={'train_batch_size':2, 'gradient_accumulation_steps':16, 'learning_rate': 4e-5, 'num_train_epochs': 3, 'max_seq_length': 100}) 

In [10]:
model.train_model(train_df)

In [12]:
result, model_outputs, wrong_predictions = model.eval_model(eval_df)

In [13]:
result, model_outputs

In [20]:
data2 = pd.read_csv(TRAIN_FILE_PATH)

In [21]:
data2 = data2.drop(['Unnamed: 0', 'title_len'], axis=1)
data2 = data2.rename(columns={'topic_idx':'labels'})
data2['labels'] = data2['labels'].astype('int')

data2

In [29]:
print(data2['title'][11111])
print(data2['labels'][11111])

In [30]:
predictions, raw_outputs = model.predict([data2['title'][11111]])

In [31]:
predictions, raw_outputs

In [ ]:
data2['labels'][0]

In [ ]:
# dataset_train = Dataset.from_pandas(data_dummies)
dataset_train = Dataset.from_pandas(data)



dataset_train, dataset_validation = dataset_train.train_test_split(test_size=0.2).values()
datasets = dataset_train, dataset_validation

datasets

In [ ]:
import torch.nn.functional as F
import torch

In [ ]:
x = torch.IntTensor(dataset_train['label'])
x = x.type(torch.int64)
# F.one_hot(x, num_classes=7)

label_onehot = F.one_hot(x, num_classes=7)
label_onehot

In [ ]:
label_onehot = label_onehot.numpy()
label_onehot

In [ ]:
test_x = torch.IntTensor(dataset_validation['label'])
test_x = test_x.type(torch.int64)
# F.one_hot(x, num_classes=7)

test_label_onehot = F.one_hot(test_x, num_classes=7)
test_label_onehot

In [ ]:
test_label_onehot = test_label_onehot.numpy()
test_label_onehot

In [ ]:
len(label_onehot)

In [ ]:
tr_title_list = np.array(dataset_train['title'])
tr_title_list

In [ ]:
len(tr_title_list)

In [ ]:
te_title_list = np.array(dataset_validation['title'])

In [ ]:
for i in range(len(label_onehot)):
    print(label_onehot[i])

In [ ]:
label_onehot[0]

In [ ]:
tr_title_list[0]

In [ ]:
num_list = []

for i in range(len(label_onehot)*2):
    if i % 2 != 0:
        num_list.append(i)
        

In [ ]:
a = [label_onehot[i].tolist() for i in range(len(label_onehot))]
b = [tr_title_list[i] for i in range(len(label_onehot))]

for i in range(len(label_onehot)):
    b.insert(num_list[i], a[i])
    
    
# b.append(a)
# b.insert()
b[27350:]

In [ ]:
len(test_label_onehot)

In [ ]:
test_a = [test_label_onehot[i].tolist() for i in range(len(test_label_onehot))]
test_b = [te_title_list[i] for i in range(len(test_label_onehot))]

for i in range(len(test_label_onehot)):
    test_b.insert(num_list[i], a[i])

In [ ]:
b[54710:]
# r_b = []
# r_b.append(b)

In [ ]:
# r_test_b = []
# r_test_b.append(test_b)

In [ ]:
import logging
import pandas as pd
from simpletransformers.classification import MultiLabelClassificationModel
import sklearn.metrics

In [ ]:
len(b)

In [ ]:
b[::2]

In [ ]:
r_b = []

for i in range(0,len(b),2):
    r_b.append(b[i:i+2])

r_b

# result = []
# for i in range(0,len(mylist),2):
#     result.append(mylist[i:i+2])
# print(result)


In [ ]:
r_test_b = []

for i in range(0,len(test_b),2):
    r_test_b.append(test_b[i:i+2])

In [ ]:
train_df = pd.DataFrame(r_b, columns=["text", "labels"])
eval_df = pd.DataFrame(r_test_b, columns=["text", "labels"])

In [ ]:
train_df

In [ ]:
# model_args = {
#     "overwrite_output_dir": True,   
# }
# model = MultiLabelClassificationModel(
#     "roberta", 
#     "distilroberta-base",
#     num_labels=7,
#     args=model_args,
# )

In [ ]:
# train_args = {
#     "evaluate_during_training_steps": 10,
#     "evaluate_during_training_verbose": True,
#     "logging_steps": 100,
#     "train_batch_size": 300,
#     "eval_batch_size": 300,
#     "num_train_epochs": 3,
# }

In [ ]:
# eval_metrics = {
# #    "f1": sklearn.metrics.f1_score,
#     "roc_auc": sklearn.metrics.roc_auc_score,
#     "avg_prc": sklearn.metrics.average_precision_score,
    
# }

In [ ]:
# model.train_model(
#     train_df,
#     eval_df=eval_df,
#     args=train_args,
#     **eval_metrics,
# )

In [ ]:
metric = load_metric("accuracy")

In [ ]:
model_checkpoint="distilroberta-base"
batch_size = 50
# task = "ynat"
num_epochs = 5
learningRate = 1e-5
weight_decay = 0.01
seed_value = 42
num_labels = 7

In [ ]:
dataset_train = Dataset.from_pandas(train_df)
dataset_validation = Dataset.from_pandas(eval_df)

datasets = dataset_train, dataset_validation

In [ ]:
datasets

In [ ]:
labels_train = dataset_train['labels']
labels_test = dataset_validation['labels']

In [ ]:
from transformers import (AutoTokenizer, 
                          AutoModelForSequenceClassification, 
                          Trainer, 
                          TrainingArguments)

In [ ]:
tokenizer = AutoTokenizer.from_pretrained("distilroberta-base")

In [ ]:
def preprocess_function(examples):
    return tokenizer(
        examples['text'],
        padding="max_length", 
        max_length = 40,
        truncation=True
    )

In [ ]:
encoded_dataset_train = dataset_train.map(preprocess_function, batched=True)
encoded_dataset_validation = dataset_validation.map(preprocess_function, batched=True)

In [ ]:
model = AutoModelForSequenceClassification.from_pretrained("distilroberta-base", num_labels=7)

In [ ]:
from sklearn.metrics import accuracy_score

def compute_metrics(pred):
    labels = pred.label_ids
    preds = pred.predictions.argmax(-1)
    acc = accuracy_score(labels, preds)
    return {
        'accuracy': acc
    }


In [ ]:
metric_name = "distilroberta-base"

args = TrainingArguments(
    output_dir='./results',
    evaluation_strategy="epoch",
    save_strategy = "epoch",
    learning_rate= learningRate,
    per_device_train_batch_size=batch_size,
    per_device_eval_batch_size=batch_size,
    num_train_epochs=num_epochs,
    weight_decay=weight_decay,
    #seed = seed_value,
    load_best_model_at_end=True,
    metric_for_best_model= metric_name,
    logging_dir='./logs',            # directory for storing logs
    logging_steps=100
)

In [ ]:
trainer = Trainer(
    model,
    args,
    train_dataset=encoded_dataset_train,
    eval_dataset=encoded_dataset_validation,
    tokenizer=tokenizer,
    compute_metrics=compute_metrics,
)

In [ ]:
# model = ClassificationModel("distilbert", 'distilbert-base-uncased', use_cuda=True, cuda_device=0, args=train_args)

In [ ]:
# model.train_model(train_df)

In [ ]:
trainer.train()

In [ ]:
from transformers import (AutoTokenizer, 
                          AutoModelForSequenceClassification, 
                          Trainer, 
                          TrainingArguments)

In [ ]:
# tokenizer = AutoTokenizer.from_pretrained("distilroberta-base")
tokenizer = AutoTokenizer.from_pretrained("roberta-base")

tokenize_func = lambda sentences: tokenizer(sentences['title'], \
                                            padding="max_length", \
                                            truncation=True)

In [ ]:
encoded_dataset_train = dataset_train.map(tokenize_func, batched=True)
encoded_dataset_validation = dataset_validation.map(tokenize_func, batched=True)

In [ ]:
model = AutoModelForSequenceClassification.from_pretrained("roberta-base", num_labels=7)

In [ ]:
from sklearn.metrics import accuracy_score

def compute_metrics(pred):
    labels = pred.label_ids
    preds = pred.predictions.argmax(-1)
    precision, recall, f1, _ = precision_recall_fscore_support(labels, preds)  # average='binary'
    acc = accuracy_score(labels, preds)
    return {
        'accuracy': acc,
        'f1': f1,
        'precision': precision,
        'recall': recall
    }

In [ ]:
batch_size = 512

training_args = TrainingArguments(
    output_dir='./results',          # output directory
    num_train_epochs=5,              # total number of training epochs
    per_device_train_batch_size=batch_size,  # batch size per device during training
    per_device_eval_batch_size=batch_size,   # batch size for evaluation
    warmup_steps=500,                # number of warmup steps for learning rate scheduler
    weight_decay=0.01,    
    # logging_dir='./logs',            
    # logging_steps=1000               # strength of weight decay
)


In [ ]:
# torch.cuda.empty.cache()

In [ ]:
trainer = Trainer(
    model=model, 
    args=training_args, 
    train_dataset=encoded_dataset_train,
    eval_dataset=encoded_dataset_validation,
    compute_metrics=compute_metrics
)

In [ ]:
trainer.train()

In [ ]:
trainer.evaluate()